In [1]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.memory import ConversationBufferMemory

import sys
# sys.path.append("../")
from configs import config
from configs.llm_provider import get_llm
from tools.paper_fetch_tools_sql import paper_fetch_toolkit
from tools.paper_analyze_tools import paper_analyze_toolkit

# 1. Initialize LLM
config.LLM_PROVIDER = 'gemini'
llm = get_llm(config)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 2. Create fetch and analyze agents
paper_fetch_agent = initialize_agent(
    tools=paper_fetch_toolkit,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

paper_analyze_agent = initialize_agent(
    tools=paper_analyze_toolkit,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

C:\Users\Xue Jiang\AppData\Local\Temp\ipykernel_8840\2534129605.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\Xue Jiang\AppData\Local\Temp\ipykernel_8840\2534129605.py:17: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  paper_

In [10]:
# 3. Define wrappers so each sub-agent can be used as a Tool
def run_fetch_agent(task: str):
    """Delegate the task to the paper fetch agent."""
    return paper_fetch_agent.run(task)

def run_analyze_agent(task: str):
    """Delegate the task to the paper analyze agent."""
    return paper_analyze_agent.run(task)

fetch_agent_tool = Tool(
    name="PaperFetcher",
    func=run_fetch_agent,
    description=(
        "Use this tool to fetch or filter a paper list for a conference. "
        "If analysis is needed, pass the resulting paper info list to PaperAnalyzer."
    )
)

analyze_agent_tool = Tool(
    name="PaperAnalyzer",
    func=run_analyze_agent,
    description=(
        "Use this tool to analyze, summarize, or answer questions about papers. "
        "If you need to analyze a paper list, fetch it first using PaperFetcher."
    )
)


In [16]:
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner


In [21]:
# 1. Create the planner (LLM decides on multi-step plan)
planner = load_chat_planner(llm)

# 2. Create the executor (agent capable of tool execution)
executor = load_agent_executor(
    llm=llm,
    tools=paper_fetch_toolkit+paper_analyze_toolkit,
    verbose=True
)

# 3. Combine into plan-and-execute agent
master_agent = PlanAndExecute(
    planner=planner,
    executor=executor,
    verbose=True
)

In [22]:
from configs.logging import configure_logging
configure_logging()  # Make sure logging is set up first


In [25]:
master_agent.run("respectively write summary of 10 privacy-related papers from NIPS 2023")



> Entering new PlanAndExecute chain...
steps=[Step(value='Search for privacy-related papers from NeurIPS 2023 proceedings.'), Step(value='Select 10 distinct papers that prominently feature privacy as a core theme (e.g., differential privacy, federated learning privacy, secure multi-party computation, privacy-preserving machine learning).'), Step(value='For each of the 10 selected papers, read its abstract and key sections (introduction, conclusion) to understand its main contribution, methodology, and findings related to privacy.'), Step(value='Write a concise summary (1-3 sentences) for each of the 10 papers, highlighting its privacy aspect.'), Step(value='Present the 10 summaries.'), Step(value='Given the above steps taken, please respond to the users original question.')]

> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "fetch_paper_list",
  "action_input": {
    "conference": "NeurIPS",
    "year": 2023
  }
}
```
Observation: {'message': 'Fetched and saved 354

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 27
}
]

In [24]:
steps=[Step(value='Access the official NeurIPS 2023 proceedings or paper list.'), Step(value='Search the paper titles and abstracts using keywords such as "privacy," "differential privacy," "federated learning," "anonymity," "confidentiality," and "privacy-preserving."'), Step(value='Review the search results to identify papers primarily focused on privacy.'), Step(value='Select 10 distinct and relevant privacy-related papers from the identified list.'), Step(value='Present the titles and, if possible, authors or links for the 10 selected papers.'), Step(value='Given the above steps taken, please respond to the users original question.')]


NameError: name 'Step' is not defined